In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy.spatial.distance import cosine
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())
import operator

0it [00:00, ?it/s]


In [2]:
with open("train.csv", "r") as f:
    train_text = f.read()
    
rows = train_text.split("\n")
rows_list = []

for i in rows:
    
    data = i.split("|")
    
    if len(data) == 3:
        user_id = data[0]
        movie_name = data[1]
        watch_time = float(data[2])
    
        rows_list.append([user_id, movie_name, watch_time])
        
train = pd.DataFrame(rows_list)
train.columns = ["user_id", "title", "watch_time"]

In [3]:
test = pd.read_csv("test.csv")

In [4]:
with open("movies_meta.csv", "r") as f:
    movies_meta = f.read()

In [5]:
movies = movies_meta.split("\n")[1:]

In [6]:
movies_df = pd.DataFrame()

movies_list = []

for movie in movies:
    
    data = movie.split("|")

    if len(data) == 4:

        movie_name = data[0]
        genre = data[1]

        langs = data[2].split(",")

        movie_length = float(data[3])

        for lang in langs:
            movies_list.append([movie_name, genre, lang, movie_length])

movies_df = pd.DataFrame(movies_list)
movies_df.columns = ["title", "movie_genre", "movie_language", "movie_length"]    

In [7]:
train_with_length = pd.merge(train, movies_df[["title", "movie_length"]], on="title")

In [8]:
train_with_length["ratio_watched"] = train_with_length["watch_time"]/train_with_length["movie_length"]

In [9]:
train_with_length = train_with_length.drop_duplicates()

In [10]:
train_matrix = train_with_length[["user_id", "title", "ratio_watched"]].pivot(index='user_id', columns='title')['ratio_watched']

In [11]:
def num_movies_watched(row):
    return len(row) - row.isnull().sum()

In [12]:
num_movies_watched_df = train_matrix.progress_apply(num_movies_watched, axis = 1)

876597it [04:20, 3371.15it/s]


In [13]:
train_matrix["num_movies"] = num_movies_watched_df

In [23]:
sparse_train_matrix = train_matrix[train_matrix.num_movies > 20]
sparse_train_matrix.drop("num_movies", axis = 1, inplace=True)

/Library/Python/2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
train_matrix.drop("num_movies", axis = 1, inplace=True)

In [12]:
train_matrix.to_pickle("train_matrix.pkl")

SystemError: error return without exception set

In [16]:
def get_cosine(i, j):
    
#     sub_df = pd.concat([sparse_train_matrix.iloc[:, i], sparse_train_matrix.iloc[:, j]], axis = 1)

    return len(sparse_train_matrix[[i, j]].dropna(axis=0, how='any'))

#     sub_df = sub_df.dropna(axis = 0, how = "all")
#     sub_df = sub_df.fillna(0)
#     return 1 - cosine(sub_df.iloc[:, 0], sub_df.iloc[:, 1])

In [27]:
item_item_sim = {}

col_names = sparse_train_matrix.columns.tolist()

res = {}

def get_similar_items(row):
    
    similar_movies = {}
    
    watched_movies = pd.DataFrame(train_matrix.iloc[train_matrix.index.tolist().index(str(row.user_id)), :].dropna()).index.tolist()
    print row.user_id, watched_movies
    
    for watched_movie in watched_movies:
        
        if watched_movie not in item_item_sim:
            
            item_item_sim[watched_movie] = {}
            
            for i in range(0, len(col_names)):                
                
                item_item_sim[watched_movie][col_names[i]] = get_cosine(watched_movie, col_names[i])
                    
        for j in item_item_sim[watched_movie]:
            
            if j not in similar_movies:
                similar_movies[j] = item_item_sim[watched_movie][j]
            else:
                similar_movies[j] += item_item_sim[watched_movie][j]
    
    
    for watched_movie in watched_movies:
        del similar_movies[watched_movie]
        
    top_similar_movies = sorted(similar_movies.items(), key=operator.itemgetter(1), reverse=True)[:20]
    
    res[str(row.user_id)] = [i[0] for i in top_similar_movies]
    
test.progress_apply(get_similar_items, axis = 1)





  0%|          | 0/6285 [00:00<?, ?it/s]

100146 ['ms dhoni: the untold story', 'nishabd']






  0%|          | 2/6285 [00:07<6:32:39,  3.75s/it]

100166 ['ms dhoni: the untold story']
100245 ['bedroom partner', 'julie']






  0%|          | 4/6285 [00:15<6:38:01,  3.80s/it]

...





100%|█████████▉| 6284/6285 [1:54:43<00:00,  2.93it/s]

99345 ['action hero biju', 'ennum eppozhum', 'kaththi', 'oppam', 'pathemaari']
99518 ['neerja']
99820 ['baahubali - the beginning', 'bhale bhale magadivoy', 'chakravarthy', 'current', 'janatha garage', 'kabali', 'khaidi no. 150', 'manmadhudu', 'naalo okkadu', 'singham - yamudu 2', 'sita rama raju']






100%|██████████| 6285/6285 [1:54:47<00:00,  1.10s/it]

99966 ['bela seshe']


0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
6255    None
6256    None
6257    None
6258    None
6259    None
6260    None
6261    None
6262    None
6263    None
6264    None
6265    None
6266    None
6267    None
6268    None
6269    None
6270    None
6271    None
6272    None
6273    None
6274    None
6275    None
6276    None
6277    None
6278    None
6279    None
6280    None
6281    None
6282    None
6283    None
6284    None
Length: 6285, dtype: object

In [86]:
cosine_vals = []
col_names = sparse_train_matrix.columns.tolist()

for i in range(0, len(train_matrix.columns.tolist())):
    if i%100 == 0:
        print i
    val = get_cosine(col_names[0], col_names[i])
#     val = get_cosine(0, i)

    cosine_vals.append(val)

0
100
200
300
400
500


KeyboardInterrupt: 

In [29]:
len(res)

6285

In [30]:
import json
with open('res_matching_counts.json', 'w') as fp:
    json.dump(res, fp)